<a href="https://colab.research.google.com/github/GillHapp/4-puppy-raffle-audit/blob/main/New_ross_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
print("✅ GPU memory cleared.")

!pip install -U peft bitsandbytes transformers accelerate


✅ GPU memory cleared.


In [ ]:
!pip install -U trl


In [ ]:
!pip install PyMuPDF


In [ ]:
import fitz


In [ ]:

def extract_text_from_pdf(pdf_path):
    text_blocks = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text = page.get_text("text").strip()
            if text:
                text_blocks.append(text)
    return text_blocks


In [ ]:
pdf_texts = extract_text_from_pdf("/content/pdfs/law1.pdf")


In [ ]:
pdf_texts


['139 STAT. 3 \nPUBLIC LAW 119–1—JAN. 29, 2025 \nPublic Law 119–1 \n119th Congress \nAn Act \nTo require the Secretary of Homeland Security to take into custody aliens who \nhave been charged in the United States with theft, and for other purposes. \nBe it enacted by the Senate and House of Representatives of \nthe United States of America in Congress assembled, \nSECTION 1. SHORT TITLE. \nThis Act may be cited as the ‘‘Laken Riley Act’’. \nSEC. 2. DETENTION OF CERTAIN ALIENS WHO COMMIT THEFT. \nSection 236(c) of the Immigration and Nationality Act (8 U.S.C. \n1226(c)) is amended— \n(1) in paragraph (1)— \n(A) in subparagraph (C), by striking ‘‘or’’; \n(B) in subparagraph (D), by striking the comma at \nthe end and inserting ‘‘, or’’; and \n(C) by inserting after subparagraph (D) the following: \n(E)(i) is inadmissible under paragraph (6)(A), (6)(C), \nor (7) of section 212(a); and \n‘‘(ii) is charged with, is arrested for, is convicted of, \nadmits having committed, or admits committi

In [ ]:
import re
def split_paragraphs(pages):
    paragraphs = []
    for page_text in pages:
        # Split on double line breaks or long newlines
        chunks = re.split(r'\n\s*\n', page_text)
        for chunk in chunks:
            clean = chunk.strip()
            if len(clean) > 30:  # ignore too short lines
                paragraphs.append(clean)
    return paragraphs


In [ ]:
paragraphs = split_paragraphs(pdf_texts)


In [ ]:
data = [{"text": p} for p in paragraphs]


In [ ]:
print(data)


[{'text': '139 STAT. 3 \nPUBLIC LAW 119–1—JAN. 29, 2025 \nPublic Law 119–1 \n119th Congress \nAn Act \nTo require the Secretary of Homeland Security to take into custody aliens who \nhave been charged in the United States with theft, and for other purposes. \nBe it enacted by the Senate and House of Representatives of \nthe United States of America in Congress assembled, \nSECTION 1. SHORT TITLE. \nThis Act may be cited as the ‘‘Laken Riley Act’’. \nSEC. 2. DETENTION OF CERTAIN ALIENS WHO COMMIT THEFT. \nSection 236(c) of the Immigration and Nationality Act (8 U.S.C. \n1226(c)) is amended— \n(1) in paragraph (1)— \n(A) in subparagraph (C), by striking ‘‘or’’; \n(B) in subparagraph (D), by striking the comma at \nthe end and inserting ‘‘, or’’; and \n(C) by inserting after subparagraph (D) the following: \n(E)(i) is inadmissible under paragraph (6)(A), (6)(C), \nor (7) of section 212(a); and \n‘‘(ii) is charged with, is arrested for, is convicted of, \nadmits having committed, or admits

In [ ]:
from datasets import Dataset  # correct import

# Suppose your data looks like this
data = [{"text": "This is a sample paragraph."}, {"text": "Another paragraph."}]

# Create Hugging Face dataset from list
dataset = Dataset.from_list(data)

print(dataset)



Dataset({
    features: ['text'],
    num_rows: 2
})


In [ ]:
dataset


Dataset({
    features: ['text'],
    num_rows: 2
})

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_fn(examples):
    tokens = tokenizer(examples["text"],truncation=True,padding="max_length",max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [ ]:
tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
tokenized


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
training_args = TrainingArguments(
    output_dir="./llama-law-domain",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    learning_rate=2e-5,
    fp16=True,
    report_to="none",
    gradient_checkpointing=True,
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install -U peft bitsandbytes transformers accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

In [ ]:
model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)


In [ ]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


In [ ]:
tokenized = dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
tokenized


Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [ ]:
!pip install -U bitsandbytes


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model,
    load_in_8bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:

args = TrainingArguments(
    output_dir="./tinyllama-lora",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized
)

In [ ]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=5, training_loss=7.138377380371094, metrics={'train_runtime': 9.5332, 'train_samples_per_second': 1.049, 'train_steps_per_second': 0.524, 'total_flos': 31814823444480.0, 'train_loss': 7.138377380371094, 'epoch': 5.0})

In [ ]:
model_path = "/content/tinyllama-lora/checkpoint-5"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")


In [ ]:
prompt = "What does the Laken Riley Act require the Secretary of Homeland Security to do when an alien has been charged with theft??"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

In [ ]:
print("\nModel Output:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Model Output:

What does the Laken Riley Act require the Secretary of Homeland Security to do when an alien has been charged with theft??
I'm not sure that the law requires anything. It says "may" and "shall". There is no requirement. The problem may be that they don't have enough information or can't verify it. Or there is a lack of resources to handle it.
It's also not clear what "charge" means in this context. It could mean that they were suspected of stealing, which would be a civil matter, or it could mean that they were charged
